In [5]:
import requests
import re
import html
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import time
import os
import random
import json

In [6]:
# Base parameters for Web Scrapping
BASE_URL = "https://www.gob.mx"
ARCHIVE_URL = f"{BASE_URL}/presidencia/es/archivo/articulos?filter_origin=archive&idiom=es&order=DESC&page="

In [7]:
def get_articles_from_page(page_num):
    """
    Extract clean titles, URLs, and dates from Gob.mx dynamic HTML.
    Args:
        page_num (int): Page number to scrape.
    Returns:
        list of dict: Each dict contains 'title', 'url', and 'date' keys
    """
    url = f"{ARCHIVE_URL}{page_num}"
    r = requests.get(url)
    r.raise_for_status()
    text = r.text

    # Extract JS-embedded HTML fragments
    fragments = re.findall(r"\$\('#prensa'\)\.append\('(.+?)'\);", text, flags=re.DOTALL)

    articles_out = []

    for frag in fragments:
        # Step 1: Decode HTML entities (e.g. &lt;, &quot;)
        frag_clean = html.unescape(frag)
        # Step 2: Replace escaped quotes \" → "
        frag_clean = frag_clean.replace('\\"', '"').replace("\\'", "'")
        # Step 3: Remove stray backslashes that break tags
        frag_clean = frag_clean.replace("\\n", "").replace("\\", "")
        # Step 4: Parse
        soup = BeautifulSoup(frag_clean, "html.parser")

        # Extract all article cards
        for art in soup.find_all("article"):
            title_el = art.find("h2")
            link_el = art.find("a", class_="small-link")
            date_el = art.find("time")

            title = title_el.get_text(strip=True) if title_el else None
            date = date_el.get_text(strip=True) if date_el else None

            # Some hrefs may end with ?idiom=es
            if link_el and link_el.has_attr("href"):
                href = link_el["href"].strip('"')
                if href.startswith("/"):
                    href = BASE_URL + href
            else:
                href = None

            if title or href:
                articles_out.append({
                    "title": title,
                    "url": href,
                    "date": date
                })
    return articles_out

In [8]:
def scrape_all_articles(max_pages=50, retries=3):
    """
    Scrape articles from multiple pages and save metadata to CSV.
    
    Args:
        max_pages (int): Maximum number of pages to scrape.

    Returns:
        pd.DataFrame: DataFrame containing all scraped articles.
    """
    # Define a list to store all data
    all_data = []
    output_dir = os.path.join("..", "data", "raw")
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "article_metadata.csv")

    # Loop through all the pages that are paginated in the site 
    for page in tqdm(range(1, max_pages + 1)):
        success = False
        for attempt in range(retries):
            try:
                # Get all the transcriptions from a single page
                page_data = get_articles_from_page(page)
                if not page_data:
                    print(f"No more data after page {page}.")
                    success = True
                    break
                all_data.extend(page_data)
                success = True
                break
            # Consider Error Codes:
            except requests.exceptions.HTTPError as e:
                print(f"HTTP error on page {page}: {e}. Retrying ({attempt+1}/{retries})...")
                time.sleep(random.uniform(5, 10))  # wait before retry
            except Exception as e:
                print(f"Other error on page {page}: {e}. Retrying...")
                time.sleep(random.uniform(5, 10))
        # If not successful, after retires
        if not success:
            print(f"Failed to fetch page {page} after {retries} retries. Skipping...")
            continue

        # Random polite delay between pages
        time.sleep(random.uniform(2, 5))

        # Save progress every 5 pages
        if page % 5 == 0:
            df_tmp = pd.DataFrame(all_data)
            df_tmp.to_csv(output_path, index=False)
            print(f"Checkpoint saved at page {page}")

    # Final save
    df = pd.DataFrame(all_data)
    df.to_csv(output_path, index=False)
    print(f"\nSaved {len(df)} articles to {output_path}")

    return df

In [ ]:
#scrape_all_articles(max_pages=59)

  8%|▊         | 5/59 [00:18<03:29,  3.88s/it]

💾 Checkpoint saved at page 5


 17%|█▋        | 10/59 [01:37<18:15, 22.36s/it]

💾 Checkpoint saved at page 10


 25%|██▌       | 15/59 [02:59<18:27, 25.17s/it]

...

 93%|█████████▎| 55/59 [06:37<00:15,  3.92s/it]

💾 Checkpoint saved at page 55


100%|██████████| 59/59 [06:55<00:00,  7.04s/it]


✅ Saved 531 articles to ../data/raw/article_metadata.csv


,title,url,date
0,Versión estenográfica. Mensaje de la President...,https://www.gob.mx/presidencia/es/articulos/ve...,"jueves, 16 de octubre de 2025Fecha de publicación"
1,Versión estenográfica. Conferencia de prensa d...,https://www.gob.mx/presidencia/es/articulos/ve...,"jueves, 16 de octubre de 2025Fecha de publicación"
2,Versión estenográfica. Conferencia de prensa d...,https://www.gob.mx/presidencia/es/articulos/ve...,"miércoles, 15 de octubre de 2025Fecha de publi..."
3,Versión estenográfica. Conferencia de prensa d...,https://www.gob.mx/presidencia/es/articulos/ve...,"martes, 14 de octubre de 2025Fecha de publicación"
4,Versión estenográfica. Conferencia de prensa d...,https://www.gob.mx/presidencia/es/articulos/ve...,"lunes, 13 de octubre de 2025Fecha de publicación"
...,...,...,...
526,Versión estenográfica. Conferencia de prensa d...,https://www.gob.mx/presidencia/es/articulos/ve...,"jueves, 03 de octubre de 2024Fecha de publicación"
527,Versión estenográfica. Conferencia de prensa d...,https://www.gob.mx/presidencia/es/articulos/ve...,"miércoles, 02 de octubre de 2024Fecha de publi..."
528,Versión estenográfica. Toma de protesta ante e...,https://www.gob.mx/presidencia/es/articulos/ve...,"martes, 01 de octubre de 2024Fecha de publicación"
529,Versión estenográfica. Mensaje de la president...,https://www.gob.mx/presidencia/es/articulos/ve...,"martes, 01 de octubre de 2024Fecha de publicación"


In [9]:
def get_transcript_structured(url):
    """
    Extract a structured transcript from a single Mañanera article page on gob.mx.

    The function retrieves the HTML of a given conference URL, parses it, and returns 
    a structured list of speech segments, each containing the speaker (if identifiable) 
    and their spoken text.

    Args:
        url (str): The full URL of the article page to scrape.

    Returns:
        list[dict]: A list of dictionaries, each with the following keys:
            - "speaker" (str or None): The name of the speaker if specified, else None.
            - "text" (str): The paragraph text, cleaned and concatenated.
    """
    # Request and parse content
    headers = {"User-Agent": "Mozilla/5.0 (compatible; MañanerasScraper/1.0)"}
    response = requests.get(url, headers=headers, timeout=15)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    # Find only the text body
    content_div = soup.find("div", class_="article-body")
    if not content_div:
        return []
    # Body text is composed of a lot of components
    entries = []
    for p in content_div.find_all("p"):
        # Get speaker (if present)
        strong = p.find("strong")
        speaker = strong.get_text(strip=True) if strong else None
        # Get text
        text = p.get_text(" ", strip=True)
        entries.append({"speaker": speaker, "text": text})
    
    return entries


In [ ]:
def get_transcript_structured(url, retries=3, backoff_factor=2):
    """
    Extract structured transcript with speaker & text, with retry logic.
    Built this function on top of the previous one to handle non success. 

    Args:
        url (str): Article URL.
        retries (int): Max number of retry attempts on failure.
        backoff_factor (int): Multiplier for exponential backoff.
    Returns:
        list[dict]: [{'speaker': ..., 'text': ...}, ...]
    """
    headers = {"User-Agent": "Mozilla/5.0 (compatible; MañanerasScraper/1.0)"}
    attempt = 0

    while attempt < retries:
        try:
            r = requests.get(url, headers=headers, timeout=20)
            r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")

            content = soup.find("div", class_="article-body")
            if not content:
                print(f"No <div class='article-body'> found for {url}")
                return []

            entries = []
            for p in content.find_all("p"):
                strong = p.find("strong")
                speaker = strong.get_text(strip=True) if strong else None
                text = p.get_text(" ", strip=True)
                if text:
                    entries.append({"speaker": speaker, "text": text})

            return entries  # success — exit function

        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            wait = backoff_factor * (2 ** attempt) + random.uniform(0, 1)
            print(f"Timeout or connection error on {url} — retry {attempt+1}/{retries} after {wait:.1f}s")
            time.sleep(wait)
            attempt += 1
            continue

        except requests.exceptions.HTTPError as e:
            print(f"HTTP error {e} on {url} — skipping")
            return []

        except Exception as e:
            print(f"Unexpected error for {url}: {e}")
            return []

    print(f"Failed after {retries} retries: {url}")
    return []


In [ ]:
df_meta = pd.read_csv("../data/raw/article_metadata.csv")
# --- Scrape all articles ---
data = []

for i, row in tqdm(df_meta.iterrows(), total=len(df_meta)):
    url = row["url"]
    if pd.isna(url):
        continue

    transcript_structured = get_transcript_structured(url)
    article = {
        "date": row["date"],
        "title": row["title"],
        "url": url,
        "transcript": transcript_structured
    }
    data.append(article)

    time.sleep(random.uniform(2, 5))

    # checkpoint every 10
    if i % 10 == 0 and i > 0:
        with open("../data/raw/article_transcripts_checkpoint.json", "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"💾 Checkpoint saved ({i} articles)")

# final save
with open("../data/processed/article_transcripts.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"Saved {len(data)} full transcripts.")

  2%|▏         | 11/531 [00:42<38:15,  4.41s/it]

💾 Checkpoint saved (10 articles)
⏳ Timeout or connection error on https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-06-de-octubre-de-2025?idiom=es — retry 1/3 after 2.8s


  3%|▎         | 14/531 [01:16<1:02:45,  7.28s/it]

⏳ Timeout or connection error on https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-02-de-octubre-de-2025?idiom=es — retry 1/3 after 2.3s
⏳ Timeout or connection error on https://www.gob.mx/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-02-de-octubre-de-2025?idiom=es — retry 2/3 after 4.8s


  3%|▎         | 17/531 [02:16<1:43:55, 12.13s/it]

...

 99%|█████████▉| 528/531 [56:29<00:13,  4.39s/it]

⏳ Timeout or connection error on https://www.gob.mx/presidencia/es/articulos/version-estenografica-toma-de-protesta-ante-el-pueblo-de-mexico?idiom=es — retry 1/3 after 2.1s


100%|██████████| 531/531 [57:05<00:00,  6.45s/it]

💾 Checkpoint saved (530 articles)


✅ Saved 531 full transcripts.
